In [1]:
from collections import defaultdict
import math
import mpmath
import numpy as np
import pandas as pd
import random
import sympy
import typing

random.seed(1)


# Goal

Compre mean square error and mean absolute error for the mildest extreme estimator, the maximum likelihood estimator, and their bias-corrected variants.


# Estimators

Functions to return estimates for population size from a list of observed fixed gene magnitudes.


In [2]:
def mildest_extrema_estimator(observed_fixed_genes: typing.List[float],) -> float:

    mildest_extreme = min(observed_fixed_genes)
    k = len(observed_fixed_genes)
    return math.log(1.0 - 0.5**(1/k)) / math.log(mildest_extreme)


In [3]:
def corrected_mildest_extrema_estimator(observed_fixed_genes: typing.List[float],) -> float:


    k = len(observed_fixed_genes)
    # expected value of estimator does not converge for k < 2,
    # just return raw estimator
    if k == 1:
        return mildest_extrema_estimator(observed_fixed_genes)

    prod_term = math.prod(
        mpmath.mpf(i) ** (
            mpmath.mpf(-1)**(i+1)
            * mpmath.mpf(math.comb(k-1, i-1))
        )
        for i in range(1, k+1)
    )
    power_term = - mpmath.mpf(0.5)**mpmath.mpf(1/k) + 1

    # correction factor from mildest_extreme_estimator_expected_value.ipynb
    correction_factor = k * mpmath.log(prod_term) * mpmath.log(power_term)

    return mildest_extrema_estimator(observed_fixed_genes) / correction_factor


In [4]:
def maximum_likelihood_estimator(observed_fixed_genes: typing.List[float],) -> float:

    k = len(observed_fixed_genes)
    return -k / sum(map(math.log, observed_fixed_genes))


In [5]:
def corrected_maximum_likelihood_estimator(observed_fixed_genes: typing.List[float],) -> float:

    k = len(observed_fixed_genes)
    # expected value of estimator does not converge for k < 2,
    # just return raw estimator
    if k == 1:
        return maximum_likelihood_estimator(observed_fixed_genes)

    # correction factor from maximum_likelihood_popsize_estimator_expected_value.ipynb
    correction_factor = k / (k - 1)

    return maximum_likelihood_estimator(observed_fixed_genes) / correction_factor


In [6]:
estimators = [
    mildest_extrema_estimator,
    corrected_mildest_extrema_estimator,
    maximum_likelihood_estimator,
    corrected_maximum_likelihood_estimator,
]


# Analytical Expressions for Mean Square Error

From [maximum_likelihood_popsize_estimator_mean_square_error.ipynb](maximum_likelihood_popsize_estimator_mean_square_error.ipynb), we have

$\begin{align*}
\mathrm{MSE}( \hat{n}_\mathrm{mle} ) = n^2 \frac{k+2}{(k-1)(k-2)}.
\end{align*}$


In [7]:
def maximum_likelihood_estimator_expected_mean_square_error(k: int, n: int,) -> float:

    if k < 3: return float('inf')

    return (
        n**2
        * (k**2 + k-2)
        / ( (k-1)**2 * (k-2) )
    )


From [mildest_extrema_popsize_estimator_mean_square_error.ipynb](mildest_extrema_popsize_estimator_mean_square_error.ipynb), we have

$\begin{align*}
\mathrm{MSE}( \hat{n}_\mathrm{mee} )
= kn\int\limits_{0}^{1} \frac{x^{n - 1} \left(1 - x^{n}\right)^{k - 1} \left(n \log{\left(x \right)} - \log{\left(2^{\frac{1}{k}} - 1 \right)} + \frac{\log{\left(2 \right)}}{k}\right)^{2}}{\log{\left(x \right)}^{2}}\, dx.
\end{align*}$


In [8]:
def mildest_extrema_estimator_expected_mean_square_error(*, k: int, n: int,) -> float:

    x = sympy.Symbol('x', nonnegative=True, real=True,)

    res = k * n * sympy.Integral(
        x**(n - 1)
        *(1 - x**n)**(k - 1)
        *(n*sympy.log(x) - sympy.log(2**(1/k) - 1) + sympy.log(2)/k)**2
        /sympy.log(x)**2,
        (x, 0, 1),
    ).evalf()
    return float(res)


In [9]:
def expected_mean_square_error(estimator_name: str, **kwargs,) -> float:
    return defaultdict(
        lambda: float('nan'),
        {
            'maximum_likelihood_estimator' : maximum_likelihood_estimator_expected_mean_square_error(**kwargs),
            'mildest_extrema_estimator' : mildest_extrema_estimator_expected_mean_square_error(**kwargs),
        },
    )[estimator_name]


# Simulated Mean Square Error


In [10]:
def sample_observations(true_popsize: int, num_observations: int,) -> typing.List[float]:
    """Simulate sampling the largest gene from within a population of `true_popsize` `num_observations` times."""

    return [
        max(random.random() for __ in range(true_popsize))
        for __ in range(num_observations)
    ]


In [11]:
def sample_squared_error(estimator, true_popsize: int, num_observations: int,) -> float:

    estimate = estimator(sample_observations(
        true_popsize,
        num_observations,
    ),)

    return (true_popsize - estimate)**2


In [12]:
records = []
for estimator in estimators:
    for k in 2, 3, 10,:
        for true_popsize in 10, 100,:

            mse = np.mean([
                sample_squared_error(
                    estimator, true_popsize, k,
                )
                for __ in range(1000)
            ])
            expected_mse = expected_mean_square_error(
                estimator.__name__,
                k=k,
                n=true_popsize,
            )

            records.append({
                'Estimator' : estimator.__name__,
                'Num Observations' : k,
                'True Population Size' : true_popsize,
                'Normalized Mean Square Error' : mse/true_popsize**2,
                'Mean Square Error' : mse,
                'Expectation-Normalized Mean Square Error' : mse/expected_mse,
                'Expected Mean Square Error' : expected_mse,
            })

res_df = pd.DataFrame.from_records(records)
res_df.round(5)


,Estimator,Num Observations,True Population Size,Normalized Mean Square Error,Mean Square Error,Expectation-Normalized Mean Square Error,Expected Mean Square Error
0,mildest_extrema_estimator,2,10,27.9154,2791.54,0.0425955,6.553600e+04
1,mildest_extrema_estimator,2,100,17.1399,171399,0.0204324,8.388608e+06
2,mildest_extrema_estimator,3,10,1.07002,107.002,0.4894,2.186398e+02
3,mildest_extrema_estimator,3,100,2.89757,28975.7,1.32527,2.186398e+04
4,mildest_extrema_estimator,10,10,0.220322,22.0322,0.902596,2.440986e+01
5,mildest_extrema_estimator,10,100,0.219163,2191.63,0.897848,2.440986e+03
6,corrected_mildest_extrema_estimator,2,10,22.3009179729835,2230.09179729835,nan,NaN
7,corrected_mildest_extrema_estimator,2,100,2.06352759861157,20635.2759861157,nan,NaN
8,corrected_mildest_extrema_estimator,3,10,0.664845816999726,66.4845816999726,nan,NaN
9,corrected_mildest_extrema_estimator,3,100,0.827132707992532,8271.32707992532,nan,NaN


# Simulated Absolute Error


In [13]:
def sample_absolute_error(estimator, true_popsize: int, num_observations: int,) -> float:

    estimate = estimator(sample_observations(
        true_popsize,
        num_observations,
    ),)

    return abs(true_popsize - estimate)


In [14]:
records = []
for estimator in estimators:
    for k in 2, 3, 10,:
        for true_popsize in 10, 100,:

            mse = np.mean([
                sample_absolute_error(
                    estimator, true_popsize, k,
                )
                for __ in range(1000)
            ])

            records.append({
                'Estimator' : estimator.__name__,
                'Num Observations' : k,
                'True Population Size' : true_popsize,
                'Mean Absolute Error' : mse,
                'Normalized Absolute Square Error' : mse/true_popsize,
            })

res_df = pd.DataFrame.from_records(records)
res_df.round(5)


,Estimator,Num Observations,True Population Size,Mean Absolute Error,Normalized Absolute Square Error
0,mildest_extrema_estimator,2,10,10.7918,1.07918
1,mildest_extrema_estimator,2,100,121.355,1.21355
2,mildest_extrema_estimator,3,10,9.10729,0.910729
3,mildest_extrema_estimator,3,100,76.0473,0.760473
4,mildest_extrema_estimator,10,10,3.47796,0.347796
5,mildest_extrema_estimator,10,100,34.7188,0.347188
6,corrected_mildest_extrema_estimator,2,10,6.86364399208046,0.686364399208046
7,corrected_mildest_extrema_estimator,2,100,82.8226297844049,0.828226297844049
8,corrected_mildest_extrema_estimator,3,10,6.1371081945439,0.61371081945439
9,corrected_mildest_extrema_estimator,3,100,60.5184472470121,0.605184472470121


# Result

The bias-corrected maximum likelihood estimator for population size shows lowest mean average error and mean square error.
